In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import json
import os

In [3]:
url = 'https://www.meet99.com/lvyou'
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
}
res = requests.get(url = url ,headers = headers)
res_soup = BeautifulSoup(res.text,'lxml')

In [4]:
sheng_list = res_soup.select('li.hasChildren')
sheng_dict = []
for item in sheng_list[1:]:
    name = item.find('a').text.split('\xa0')[0]
    href = 'https://www.meet99.com' +  item.find('a')['href']
    sheng_dict.append({
        'name':name,
        'url':href
    })

In [6]:
sheng_city_list = []
for _ in tqdm(sheng_dict):
    res = requests.get(url = _['url'] ,headers = headers)
    time.sleep(1)
    res_soup = BeautifulSoup(res.text,'lxml')
    # 找市
    shi = res_soup.find('li',class_='open').find('li',class_='open').select('ul > li')
    for item in shi:
        name = item.text
        if name.strip() != '':
            name = name.split('\xa0')[0]
            url = 'https://www.meet99.com' +   item.find('a')['href']
            sheng_city_list.append({
                'sheng':_['name'],
                'sheng_url':_['url'],
                'city_name':name,
                'city_url':url
            })

100%|██████████| 34/34 [00:39<00:00,  1.16s/it]


# 得到省 城市 地址 的信息

In [7]:
sheng_city_list

[{'sheng': '北京市',
  'sheng_url': 'https://www.meet99.com/lvyou-beijing.html',
  'city_name': '东城区',
  'city_url': 'https://www.meet99.com/lvyou-dongcheng.html'},
 {'sheng': '北京市',
  'sheng_url': 'https://www.meet99.com/lvyou-beijing.html',
  'city_name': '西城区',
  'city_url': 'https://www.meet99.com/lvyou-xicheng.html'},
 {'sheng': '北京市',
  'sheng_url': 'https://www.meet99.com/lvyou-beijing.html',
  'city_name': '朝阳区',
  'city_url': 'https://www.meet99.com/lvyou-chaoyang.html'},
 {'sheng': '北京市',
  'sheng_url': 'https://www.meet99.com/lvyou-beijing.html',
  'city_name': '丰台区',
  'city_url': 'https://www.meet99.com/lvyou-fengtai.html'},
 {'sheng': '北京市',
  'sheng_url': 'https://www.meet99.com/lvyou-beijing.html',
  'city_name': '石景山区',
  'city_url': 'https://www.meet99.com/lvyou-shijingshan.html'},
 {'sheng': '北京市',
  'sheng_url': 'https://www.meet99.com/lvyou-beijing.html',
  'city_name': '海淀区',
  'city_url': 'https://www.meet99.com/lvyou-haidian.html'},
 {'sheng': '北京市',
  'sheng_url':

In [10]:
json.dump(
    sheng_city_list,
    open('sheng_city_list.json','w'),
    indent=4,
    ensure_ascii=False
)

# 开始访问每个城市的景点
#一共443个城市





In [7]:
sheng_city_list = json.load(open('sheng_city_list.json','r'))
print(sheng_city_list[0].keys())
len(sheng_city_list)# 一共 443个城市

dict_keys(['sheng', 'sheng_url', 'city_name', 'city_url'])


443

In [12]:
sheng_city_jingqu_list = json.load(open('sheng_city_jingqu_list.json','r'))
print(len(sheng_city_jingqu_list))
#    3089

3192


In [10]:
if os.path.exists('sheng_city_jingqu_list.json'):
    sheng_city_jingqu_list = json.load(open('sheng_city_jingqu_list.json','r'))
else:
    sheng_city_jingqu_list=[]
has_scrapy = [ ] # 爬过的列表

for item in sheng_city_jingqu_list:
    has_scrapy.append(item['city_url'])

for _ in tqdm(sheng_city_list):
#     url = 'https://www.meet99.com/lvyou-dongcheng.html'
    this_url = _['city_url']
    if this_url in has_scrapy:# 爬过了就不需要爬取了
#         print("?")
        continue

    res = requests.get(url = this_url ,headers = {
        "User-Agent":get_random_browser()
    })

    time.sleep(5)
    res_soup = BeautifulSoup(res.text,'lxml')
    
    boxs = res_soup.find_all('li',class_='box') # 城市的每个景区的box
    for box in boxs:
        try:
            jingqu_name = box.find('div',class_='img').find('a').text
            jingqu_url =  'https://www.meet99.com' + box.find('div',class_='img').find('a')['href']
            ever = box.find('a',class_='ever').find('span').text
            never= box.find('a',class_='never').find('span').text
            if box.find('div',class_='cornernum'):
                level = box.find('div',class_='cornernum').text
            else:
                level = ''
            sheng_city_jingqu_list.append({
                'sheng':_['sheng'],
                'sheng_url':_['sheng_url'],
                'city_name':_['city_name'],
                'city_url':_['city_url'],
                
                'jingqu_name':jingqu_name,
                'jingqu_url':jingqu_url,
                'ever':ever,
                'never':never,
                'level':level,
            })
            json.dump(
            sheng_city_jingqu_list,
            open('sheng_city_jingqu_list.json','w'),
            indent=4,
            ensure_ascii=False)
           
            
        except Exception as e:
            
            print(box)
            print(e)
            continue
json.dump(
    sheng_city_jingqu_list,
    open('sheng_city_jingqu_list.json','w'),
    indent=4,
    ensure_ascii=False
)

 24%|██▎       | 105/443 [00:08<00:27, 12.41it/s]

<li class="box" style="height:227px;"><div style="width:180px;height:180px;margin:0 auto;margin-top:6px;">
<script src="//b1.meet99.com/site/resource/js/f4jxvz.js?kfx=bnxexb" type="text/javascript"></script></div></li>
'NoneType' object has no attribute 'find'
<li class="box" style="height:227px;"><div style="width:180px;height:180px;margin:0 auto;margin-top:6px;">
<script src="//b1.meet99.com/site/resource/js/f4jxvz.js?kfx=bnxexb" type="text/javascript"></script></div></li>
'NoneType' object has no attribute 'find'


 28%|██▊       | 125/443 [00:16<00:56,  5.61it/s]

<li class="box" style="height:227px;"><div style="width:180px;height:180px;margin:0 auto;margin-top:6px;">
<script src="//b1.meet99.com/site/resource/js/f4jxvz.js?kfx=bnxexb" type="text/javascript"></script></div></li>
'NoneType' object has no attribute 'find'


 29%|██▊       | 127/443 [00:25<07:25,  1.41s/it]

<li class="box" style="height:227px;"><div style="width:180px;height:180px;margin:0 auto;margin-top:6px;">
<script src="//b1.meet99.com/site/resource/js/f4jxvz.js?kfx=bnxexb" type="text/javascript"></script></div></li>
'NoneType' object has no attribute 'find'


 29%|██▉       | 128/443 [00:43<01:47,  2.93it/s]


ProxyError: HTTPSConnectionPool(host='www.meet99.com', port=443): Max retries exceeded with url: /lvyou-zhenjiang.html (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

In [124]:
url = 'https://www.meet99.com/lvyou-dongcheng.html'
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
}
res = requests.get(url = url ,headers = {
    "User-Agent":get_random_browser()
})

<Response [200]>

In [126]:
res.status_code

200

In [9]:
import random
import requests
import json
def get_random_browser():
    """
    useragent与浏览器访问选择"""
    with open("fake-useragent.json", "r") as f:
        browsers_json = json.load(f)
        browsers_json = json .loads(browsers_json)
        browsers = browsers_json ["browsers"]
    #随机出来一个浏览器类型
        i = random.randint(0, len(browsers))
        browser_name = ""
        if i == 0:
            browser_name = "chrome"
        elif i == 1:
            browser_name = "opera"
        elif i == 2:
            browser_name = "firefox"
        elif i == 3:
            browser_name = "internetexplorer"
        else:
            browser_name = "safari"
        final_browser = browsers [browser_name] [random.randint(0, len(browsers [browser_name])-1)]
        return final_browser
get_random_browser()
# r = requests.post(url='',headers=headers)
# r.encoding = r.apparent_encoding


'Mozilla/5.0 (Windows; U; Windows NT 6.1; fr-FR) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27'

# 下面开始爬取每个景区的信息

In [151]:
url = 'https://www.meet99.com/jingdian-tiantan.html'
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
}
res = requests.get(url = url ,headers = headers)
res_soup = BeautifulSoup(res.text,'lxml')

/Users/czq/opt/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [152]:
res_soup

In [20]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys #Keys类   #common
from selenium.webdriver.support.ui import Select #select类定位下拉框
drvier=webdriver.Chrome()
url='https://www.meet99.com/jingdian-ForbiddenCity.html'#携程网
drvier.get(url)
one_soup = BeautifulSoup(drvier.page_source,'lxml')
time.sleep(1)

In [21]:
one_soup

<html><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><title>2021北京故宫旅游攻略-故宫自助游-故宫门票交通天气景点介绍</title><meta content="提供2021最新北京故宫的旅游攻略，包括北京故宫景点的详细介绍，门票价格、交通天气、游玩攻略、驴友照片等。" name="description"/><meta content="最新北京故宫旅游攻略,北京故宫图片,北京故宫门票,北京故宫地图,北京故宫自助游" name="keywords"/><meta content="webkit" name="renderer"/><link href="https://m.meet99.com/jingdian-ForbiddenCity.html" media="only screen and (max-width: 640px)" rel="alternate"/><meta content="format=html5;url=https://m.meet99.com/jingdian-ForbiddenCity.html" name="mobile-agent"/><meta content="format=xhtml;url=https://m.meet99.com/jingdian-ForbiddenCity.html" name="mobile-agent"/><link href="https://s.meet99.cn/newimg/logo3.png" rel="apple-touch-icon-precomposed"/><link href="/favicon.ico" rel="shortcut icon" type="image/ico"/><link href="https://s.meet99.cn/css/sngeneric.css?v=20210413.css" rel="stylesheet" type="text/css"/><style>#content{position:relative;}
#jdleft{margin:0px 366px 0 1px;border-bottom:0;}
#jdright

In [30]:
## 获取图片地址
img_url = one_soup.find('div',class_='img')['style'].split('(')[1].split(')')[0]
img_url = img_url.replace('"','')
print(img_url)

https://i.meet99.cn/image/d9/b8/fe3229aa56cd2dc605c5244d4cc0.jpg


In [29]:
# 获取title
title = one_soup.find('h1',class_='title').text
print(title)

北京故宫旅游攻略（1/共84页）


In [46]:
# detail获取介绍
detail = one_soup.find('div',class_='spotinfo')
print(detail)

None


In [37]:
# 获取下一页
one_soup.find('div',id='jdright').find('a',string='下一页')

<a href="/jingdian-ForbiddenCity-1992.html" style="color:#66F;font-weight:bold;" t="前朝区">下一页</a>

In [16]:
import requests
'''代理IP地址（高匿）'''
proxy = {
  'http': 'http://221.216.138.178:9000',
  'https': 'https://221.216.138.178:9000'
}
'''head 信息'''
head = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36', 
       'Connection': 'keep-alive'}
'''http://icanhazip.com会返回当前的IP地址'''
p = requests.get('http://icanhazip.com', headers=head, proxies=proxy)
print(p.text)



<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<title>ERROR: Gateway Timeout</title></head>
<body><h2>ERROR: Gateway Timeout</h2>
<hr noshade size="1px"/>

<p>While trying to retrieve the URL <a href="http://icanhazip.com/">http://icanhazip.com/</a>:</p>
<ul><li><strong>Connection timed out</strong></li></ul>

<p>Your cache administrator is <a href="mailto:webmaster">webmaster</a>.
<br clear="all"/><hr noshade size="1px"/>
<address>Generated Sat, 06 Nov 2021 13:37:03 GMT by 221.219.99.254 (Mikrotik HttpProxy)</address>
</body></html>

